In [1]:
from tensorflow.python.client import device_lib 

# check if GPU is available
devices = [x.name for x in device_lib.list_local_devices() if x.device_type == 'GPU']

In [2]:
import os
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import tensorflow as tf
tf.enable_eager_execution()

# train using GPU
if len(devices) > 0:
    tf.device('/gpu:0')
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

In [3]:
X = []
y_label = []
for finger_idx in [1,2,3,4]:
    for file_idx in range(1,1001):   
        path = "data/finger%d/finger%d_%d.jpg" %(finger_idx, finger_idx, file_idx)
        temp = load_img(path, target_size=(224, 224))
        keep = temp.copy()
        X.append(img_to_array(keep))
        temp.close()
        y_label.append(finger_idx-1)
X = np.array(X)
y_label = np.array(y_label)

In [4]:
# Split into train, test
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y_label, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)

# Train Model

In [9]:
# Define CNN model
model = tf.keras.models.Sequential([
            tf.keras.layers.Conv2D(32, kernel_size=(3,3), activation="relu", input_shape=(224,224,3), padding="same"),
            tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
            tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation="relu", padding="same"),
            tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
#             tf.keras.layers.Conv2D(128, kernel_size=(3,3), activation="relu", padding="same"),
#             tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
            tf.keras.layers.Flatten(),
#             tf.keras.layers.Dense(1024, activation="relu"),
#             tf.keras.layers.Dropout(0.2),
#             tf.keras.layers.Dense(512, activation="relu"),
            tf.keras.layers.Dense(64, activation="relu"),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(4, activation="softmax")])

In [10]:
# optimizer
optimizer = tf.train.AdadeltaOptimizer(learning_rate=0.2)
# compile
model.compile(optimizer=optimizer, 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

In [11]:
callbacks = [EarlyStopping(monitor='val_loss', min_delta=0.001, patience=10, mode='auto', verbose=1,)]
train_gen = ImageDataGenerator(rescale = 1./255,rotation_range=20,width_shift_range=0.2,height_shift_range=0.2,horizontal_flip=True)
train_gen.fit(X_train)
val_gen = ImageDataGenerator(rescale = 1./255,rotation_range=20,width_shift_range=0.2,height_shift_range=0.2,horizontal_flip=True)
val_gen.fit(X_val)

In [12]:
batch_size = 100
model.fit_generator(train_gen.flow(X_train, y_train, batch_size=batch_size), 
                    steps_per_epoch=len(X_train)/batch_size, 
                    epochs=50, 
                    validation_data=val_gen.flow(X_val, y_val, batch_size=batch_size), 
                    callbacks=callbacks, verbose=1)

Epoch 1/50
29/28 [==============================] - 34s 1s/step - loss: 1.4602 - acc: 0.3149 - val_loss: 1.3434 - val_acc: 0.4653
Epoch 2/50
29/28 [==============================] - 27s 937ms/step - loss: 1.3374 - acc: 0.3850 - val_loss: 1.3426 - val_acc: 0.2722
Epoch 3/50
29/28 [==============================] - 27s 937ms/step - loss: 1.2855 - acc: 0.4229 - val_loss: 1.2620 - val_acc: 0.4264
Epoch 4/50
29/28 [==============================] - 28s 950ms/step - loss: 1.2427 - acc: 0.4388 - val_loss: 1.1920 - val_acc: 0.4750
Epoch 5/50
29/28 [==============================] - 27s 939ms/step - loss: 1.1986 - acc: 0.4461 - val_loss: 1.1263 - val_acc: 0.5014
Epoch 6/50
29/28 [==============================] - 28s 960ms/step - loss: 1.1563 - acc: 0.4529 - val_loss: 1.1199 - val_acc: 0.5042
Epoch 7/50
29/28 [==============================] - 28s 952ms/step - loss: 1.1244 - acc: 0.4643 - val_loss: 1.0761 - val_acc: 0.5264
Epoch 8/50
29/28 [==============================] - 29s 995ms/step - los

In [9]:
model.evaluate(X_test, y_test)

13/13 [==============================] - 1s 76ms/step


[5.848296604156494, 0.635]

In [9]:
import json
import tensorflowjs as tfjs

tfjs.converters.save_keras_model(model, 'js_model/')

Using TensorFlow backend.
